# Layer: Gold (Business)
**Project:** Lean Logistics Data Pipeline  
**Business Domain:** E-commerce (Olist Dataset)\
**Table Name:** `obt_sales`

---
## 📑 Notebook Information
| Version | Date | Author | Summary of Changes |
| :--- | :--- | :--- | :--- |
| v1.0 | 2026-02-20 | Tássia Marchito | Initial creation of the One Big Table (OBT) for Sales. |

---
## 🎯 Objectives
This notebook creates a flattened, denormalized table (OBT) to provide a seamless experience for BI tools (Power BI, Tableau, etc.).
* **Denormalization:** Joining `ft_sales` with all dimensions (`dm_products`, `dm_customers`, `dm_sellers`).
* **Performance:** Optimized for analytical queries without the need for complex joins.
* **Governance:** 100% column documentation and discovery tags for Unity Catalog.

In [0]:
from pyspark.sql.functions import col, current_timestamp

In [0]:
# 1. Configurações
gold_db = "cat_tm_services_gold.db_logistics"
target_table = f"{gold_db}.obt_sales"

print(f"🚀 Building OBT: {target_table}...")

# 2. Carregamento das Tabelas Gold
df_fact = spark.read.table(f"{gold_db}.ft_sales")
df_prod = spark.read.table(f"{gold_db}.dm_products")
df_cust = spark.read.table(f"{gold_db}.dm_customers")
df_sell = spark.read.table(f"{gold_db}.dm_sellers")

# 3. Join para Denormalização Total
# Usamos left joins para garantir que nenhuma venda seja perdida caso falte algum dado dimensional
df_obt = df_fact.join(df_prod, df_fact.id_product == df_prod.cd_product_id, "left") \
                .join(df_cust, df_fact.id_customer == df_cust.cd_customer_id, "left") \
                .join(df_sell, df_fact.id_seller == df_sell.cd_seller_id, "left")

# 4. Seleção Final (Limpando colunas repetidas de join e organizando)
df_obt_final = df_obt.select(
    # Order Info
    df_fact["id_order"],
    df_fact["ts_order_purchase"],
    df_fact["ts_order_delivered_customer"],
    df_fact["dt_order_estimated_delivery"],
    # Product Info
    df_fact["id_product"],
    df_prod["ds_product_category"],
    # Customer Info
    df_fact["id_customer"],
    df_cust["nm_city"].alias("nm_customer_city"),
    df_cust["nm_state"].alias("nm_customer_state"),
    # Seller Info
    df_fact["id_seller"],
    df_sell["nm_city"].alias("nm_seller_city"),
    df_sell["nm_state"].alias("nm_seller_state"),
    # Metrics
    df_fact["vl_price"],
    df_fact["vl_freight_value"],
    df_fact["vl_total_order"],
    df_fact["vl_review_score"],
    df_fact["nr_days_to_deliver"],
    df_fact["nr_days_delivery_performance"]
).withColumn("ts_gold_at", current_timestamp())

# 5. Escrita da OBT
df_obt_final.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(target_table)

# 6. Governança e Metadados
print(f"📝 Applying tags and documentation to {target_table}...")

spark.sql(f"ALTER TABLE {target_table} SET TAGS ('quality' = 'gold', 'domain' = 'logistics', 'type' = 'obt')")

# Dicionário de Dados (Resumido para os principais campos de negócio)
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN id_order COMMENT 'Unique identifier for the order'")
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN ds_product_category COMMENT 'Category of the product in English'")
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN nm_customer_city COMMENT 'City where the customer is located'")
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN vl_total_order COMMENT 'Total revenue from the order'")
spark.sql(f"ALTER TABLE {target_table} ALTER COLUMN nr_days_to_deliver COMMENT 'Lead time in days for delivery'")

print(f"✅ OBT {target_table} is ready for BI consumption!")